In [1]:
import pandas as pd
import sqlite3

In [2]:
connect = sqlite3.connect('../data/checking-logs.sqlite')

In [3]:
deadlines_head = pd.io.sql.read_sql('select * from test limit 10', connect)
deadlines_head

,index,uid,labname,first_commit_ts,first_view_ts
0,0,user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
1,1,user_1,laba04s,2020-04-26 17:12:11.843671,2020-04-26 21:53:59.624136
2,2,user_1,laba05,2020-05-02 19:15:18.540185,2020-04-26 21:53:59.624136
3,3,user_1,laba06,2020-05-17 16:26:35.268534,2020-04-26 21:53:59.624136
4,4,user_1,laba06s,2020-05-20 12:23:37.289724,2020-04-26 21:53:59.624136
5,5,user_1,project1,2020-05-14 20:56:08.898880,2020-04-26 21:53:59.624136
6,6,user_10,laba04,2020-04-25 08:24:52.696624,2020-04-18 12:19:50.182714
7,7,user_10,laba04s,2020-04-25 08:37:54.604222,2020-04-18 12:19:50.182714
8,8,user_10,laba05,2020-05-01 19:27:26.063245,2020-04-18 12:19:50.182714
9,9,user_10,laba06,2020-05-19 11:39:28.885637,2020-04-18 12:19:50.182714


## средняя разница между всеми пользователями за период времени до того, как каждый из них совершил свой первый визит на страницу, и после для test 

In [22]:
query_test = """
    WITH diffs AS (
        SELECT
            t.uid,
            t.labname,
            t.first_commit_ts,
            t.first_view_ts,
            CAST((strftime('%s', t.first_commit_ts) - strftime('%s', datetime(d.deadlines, 'unixepoch'))) / 3600.0 AS INTEGER) AS delta_hours,
            CASE
                WHEN t.first_commit_ts < t.first_view_ts THEN 'before'
                ELSE 'after'
            END AS time_period
        FROM 
            test t
        JOIN 
            deadlines d ON t.labname = d.labs
        WHERE 
            t.labname != 'project1'),
    users AS (    
        SELECT uid, COUNT(DISTINCT time_period) AS cnt_periods
        FROM diffs
        GROUP BY uid),
    filt_diff AS (
        SELECT d.*
        FROM diffs d
        JOIN users uc ON d.uid = uc.uid
        WHERE uc.cnt_periods = 2),
    avg_user AS (
        SELECT 
            uid, 
            time_period,
            AVG(delta_hours) AS avg_diff
        FROM filt_diff
        GROUP BY time_period, uid
        )
        SELECT 
            time_period,
            AVG(avg_diff) AS avg_diff
        FROM avg_user
        GROUP BY time_period
"""
test_results = pd.io.sql.read_sql(query_test, connect)
test_results

,time_period,avg_diff
0,after,-99.523810
1,before,-66.047619


## То же самое для control

In [23]:
query_control = """
    WITH diffs AS (
        SELECT
            t.uid,
            t.labname,
            t.first_commit_ts,
            t.first_view_ts,
            CAST((strftime('%s', t.first_commit_ts) - strftime('%s', datetime(d.deadlines, 'unixepoch'))) / 3600.0 AS INTEGER) AS delta_hours,
            CASE
                WHEN t.first_commit_ts < t.first_view_ts THEN 'before'
                ELSE 'after'
            END AS time_period
        FROM 
            control t
        JOIN 
            deadlines d ON t.labname = d.labs
        WHERE 
            t.labname != 'project1'),
    users AS (    
        SELECT uid, COUNT(DISTINCT time_period) AS cnt_periods
        FROM diffs
        GROUP BY uid),
    filt_diff AS (
        SELECT d.*
        FROM diffs d
        JOIN users uc ON d.uid = uc.uid
        WHERE uc.cnt_periods = 2),
    avg_user AS (
        SELECT 
            uid, 
            time_period,
            AVG(delta_hours) AS avg_diff
        FROM filt_diff
        GROUP BY time_period, uid
        )
        SELECT 
            time_period,
            AVG(avg_diff) AS avg_diff
        FROM avg_user
        GROUP BY time_period
"""
control_results = pd.io.sql.read_sql(query_control, connect)
control_results

,time_period,avg_diff
0,after,-99.322222
1,before,-98.033333


In [7]:
connect.close()